Extract mel-spectrogram from audio

In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing import image
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, TimeDistributed, LSTM, Input, Dropout, SimpleRNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss
from tensorflow.keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score

import os
import glob
import matplotlib.pyplot as plt


Find graphic specs

train

In [2]:
# Load the ResNet50 model
base_model = ResNet50(include_top=False, pooling='avg')

# Make a new model that outputs right before the classification layer
model = Model(inputs=base_model.input, outputs=base_model.output)

In [3]:
def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 224, 224)
    x = np.expand_dims(x, axis=0)  # this is a Numpy array with shape (1, 3, 224, 224)
    x = preprocess_input(x)  # this is a Numpy array with shape (1, 224, 224, 3)
    features = model.predict(x)[0]
    return features


In [4]:
image_files_HC = glob.glob(os.path.join("plots", "HC", "*.png"))
image_files_PD = glob.glob(os.path.join("plots", "PD", "*.png"))
image_files = image_files_HC + image_files_PD

# Extract the features
features = []
labels = []
for image_file in image_files:
    features.append(extract_features(image_file, model))
    if 'HC' in image_file:
        labels.append(0)
    else:
        labels.append(1)

# Convert to arrays
features = np.array(features)
labels = np.array(labels)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


1/1 [==============================] - 0s 63ms/step


In [5]:
# Train the XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)

preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])
# Calculate accuracy
accuracy = accuracy_score(y_test, best_preds)
print('Accuracy: {:.2f}'.format(accuracy * 100))

# Calculate precision
precision = precision_score(y_test, best_preds)
print('Precision: {:.2f}'.format(precision * 100))

# Calculate recall
recall = recall_score(y_test, best_preds)
print('Recall: {:.2f}'.format(recall * 100))

Accuracy: 80.00
Precision: 82.35
Recall: 73.68
